# Azure Machine Learning Engineer
## Project 3 - Capstone AutoML

## Create a workspace
In this step we are making an Azure Workspace and setting up an experiment

In [1]:
# Create a new workspace and define an experiment.

from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.get_details()

# Choose a name for the experiment
experiment_name = 'udacity-project-automl'
experiment = Experiment(workspace=ws, name = experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-146639
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-146639


In [2]:
# Create a compute cluster to provision VM Resources.

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = 'cpu-cluster-01'

#Verify that the culster does not exist already
try:
    compute_target = ComputeTarget(workspace = ws, name = cpu_cluster_name)
    print('Found existing cluster, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Creating.......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

The below information is for a locally (Azure) hosted dataset.  In this case a CSV was uploaded to the datablob store and to allow access you interogate it and go to the consume tab and copy the below code

In [23]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required

from azureml.core import Workspace, Dataset

subscription_id = 'd4ad7261-832d-46b2-b093-22156001df5b'
resource_group = 'aml-quickstarts-146639'
workspace_name = 'quick-starts-ws-146639'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Adult')

# create a pandas dataframe from the tabular dataset for use with sklearn functions.
df = dataset.to_pandas_dataframe()

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.dataset_factory import DataType

#file uploaded to github
url = 'https://raw.githubusercontent.com/boffyd/UdacityMLOPs-Capstone/main/adult.csv'

# pass url to Tabular dataset.  Note this is different to pandas dataframe, and gets converted to a dataframe in the function.

dataset = TabularDatasetFactory.from_delimited_files(url,header = True)
ds = dataset.to_pandas_dataframe().dropna()

In [6]:
tabular_dataset = dataset.filter(dataset['native-country'].contains('United-States'))

Method filter: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [7]:
tabular_dataset.take(3).to_pandas_dataframe()

,Column1,age,workclass,fnlwgt,education,education-num.,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [8]:
tabular_dataset = tabular_dataset.drop_columns(['Column1',
                                                'fnlwgt',
                                                'education-num',
                                                'race'])

In [9]:
tabular_dataset.take(3).to_pandas_dataframe()

,age,workclass,education,education-num.,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    compute_target = compute_target,
    primary_metric = 'accuracy',
    n_cross_validations = 3, #typically 5, used to make time go quicker.
    task = 'classification',
    training_data = tabular_dataset,
    debug_log = 'automl_errors.log',
    label_column_name = 'wage',
    enable_onnx_compatible_models = True)

In [11]:
# Submit your automl run
experiment = Experiment(ws, 'udacity-project-automl')
automl_run = experiment.submit(config = automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster-01 with default configuration
Running on remote compute: cpu-cluster-01


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_0cd3151f-aa61-46db-8ff7-6c623baed449,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

In [12]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

### Best Model

In [15]:
import azureml.train.automl

import azureml.automl.core
from azureml.automl.runtime.onnx_convert import OnnxConverter

best_automl_run, best_automl_onnx_model = automl_run.get_output(return_onnx_model = True)
OnnxConverter.save_onnx_model(best_automl_onnx_model, file_path = 'outputs/best_automl_model.onnx')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

In [20]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.


# two sets of data to score
data = {"data":
        [
          {
            "age": 40,
            "workclass": "private",
            "education": "Bachelors",
            "education-num.": 12,
            "marital-status": "Divorced",
            "occupation": "Exec-managerial",
            "relationship": "Unmarried",
            "sex": "Male",
            "capital-gain": 2000,
            "capital-loss": 0,
            "hours-per-week": 45,
            "native-country": "United-States"
          },
          {
            "age": 28,
            "workclass": "state-gove",
            "education": "HS-grad",
            "education-num.": 9,
            "marital-status": "Never-married",
            "occupation": "Machine-op-inspct",
            "relationship": "Unmarried",
            "sex": "Male",
            "capital-gain": 2000,
            "capital-loss": 0,
            "hours-per-week": 45,
            "native-country": "United-States"
          },
      ]
    }


body = str.encode(json.dumps(data))

url = 'http://6f92a511-6247-4ca0-8e68-0fb9caa1b2b9.southcentralus.azurecontainer.io/score'
api_key = 'Ptv8wKeDcTKCod1qO02pyaJRxZqksGf2' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'"{\\"result\\": [\\" <=50K\\", \\" <=50K\\"]}"'
